In [37]:
from keycloak import KeycloakAdmin, KeycloakOpenIDConnection

In [42]:
# INIT INSTANCE KeycloakOpenIDConnection

kc_admin = KeycloakAdmin(
    server_url="http://0.0.0.0:8088",
    username="auth",
    password="admin",
    realm_name="auth",
    client_id="account",
    client_secret_key="bWUrxqhCIrBFwZAMTAllEJRg0bZebGRT",
)

In [43]:
# INIT INSTANCE KeycloakAdmin

# kc_admin = KeycloakAdmin(connection=conn)

In [44]:
import string
import random
import json


def generate_id_number():
    allowed_chars = string.ascii_uppercase + string.digits
    unique_chars = random.sample(
        population=[char for char in allowed_chars if char not in "O0LI"], k=8
    )
    random.shuffle(unique_chars)
    matricule = "-".join(
        [
            "".join(unique_chars[:2]),
            "".join(unique_chars[2:5]),
            "".join(unique_chars[5:]),
        ]
    )
    return matricule


payload_user = {
    "username": "emanuel.doe@pm.me",
    "email": "emanuel.doe@pm.me",
    "firstName": "John",
    "lastName": "Doe",
    "enabled": True,
    "emailVerified": True,
    "credentials": [{"type": "password", "value": "p@55word"}],
    "attributes": {
        "id_number": generate_id_number(),
        "phone_number": "+2250151571396",
        "primary": True,
        "app_users": True,
    },
}

new_user = await kc_admin.a_create_user(payload=payload_user)
new_user

'0115b35c-1034-4156-bd61-157828611740'

In [ ]:
# GET ALL USERS

users = await kc_admin.a_get_users({"first": 1, "max": 10})
users

In [ ]:
# GET COUNT USERS

users_count = await kc_admin.a_users_count({})
users_count

In [ ]:
# GET USER

user = await kc_admin.a_get_user(user_id=new_user)
user

In [ ]:
# CREATE GROUP

import string
import random
from slugify import slugify


for _ in range(1, 5):
    c = random.choice(string.ascii_uppercase)
    group_name = f"agent rencenseur groupe {c}".upper()
    payload = {"name": group_name, "attributes": {"slug": [slugify(group_name)]}}

    new_group = await kc_admin.a_create_group(payload=payload, skip_exists=True)
    new_group

In [ ]:
# GET ALL GROUPS

groups = await kc_admin.a_get_groups({})
groups

In [ ]:
# GET GROUP

sliced_groups = groups[3:]
group_id = sliced_groups[0].get("id")
group = await kc_admin.a_get_group(group_id=group_id)
group

In [ ]:
# ASSIGN GROUP TO USER

user_id = users[0].get("id")
user_groups = await kc_admin.a_get_user_groups(user_id=user_id)

if user_groups:
    await kc_admin.a_group_user_remove(
        user_id=user_id, group_id=user_groups[0].get("id")
    )

result = await kc_admin.a_group_user_add(user_id=user_id, group_id=group["id"])
result

In [ ]:
# GET USER GROUPS

groups = await kc_admin.a_get_user_groups(user_id=user_id)
groups